In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
sns.set()
plt.rcParams['font.sans-serif'] = ['SimHei'] # 显示中文标题 
plt.rcParams['axes.unicode_minus'] = False # 正常显示正负号

* 问题：1：在总表里筛选数据，接着创建唯一ID，最后把其他子表中总表没有的内容根据唯一ID拼接到总表后面
* 过滤：空值过滤(删整行),ST(不景气的)过滤,范围过滤(筛选2.5~97.5%间的连续数据,避免极端影响)
# 1.读数据

In [2]:
df1 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\roa.xlsx')
df2 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\独董比例.xlsx')
df3 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\固定资产.xlsx')
df4 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\营收增长率.xlsx')
df5 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\资产负债率.xlsx')
df6 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\总资产----同时也是班级的表，其他五个是年纪的表.xlsx')

In [3]:
df6.head()

,Stkcd,Reptdt,Stknme,Nindcd,Nindnme,A001000000
0,证券代码,统计截止日期,证券简称,行业代码,行业名称,总资产
1,000004,2014-12-31,国农科技,C27,医药制造业,3.38283e+08
2,000004,2015-12-31,国农科技,C27,医药制造业,3.98674e+08
3,000004,2016-12-31,国农科技,C27,医药制造业,2.23716e+08
4,000004,2017-12-31,国农科技,C27,医药制造业,2.68844e+08


# 2.筛选12.31和A数据

In [4]:
def myfitter(col, ends, df):
    ###要过滤的列,过滤内容,df###
    return df[[t.endswith(ends) for t in df.iloc[:,col].values]]

In [5]:
df1 = myfitter(1,'12-31',df1)
df2 = myfitter(1,'12-31',df2)
df3 = myfitter(1,'12-31',df3)
df4 = myfitter(1,'12-31',df4)
df5 = myfitter(1,'12-31',df5)
df6 = myfitter(1,'12-31',df6)

df1 = myfitter(2,'A',df1)
df3 = myfitter(2,'A',df3)
df4 = myfitter(2,'A',df4)
df5 = myfitter(2,'A',df5)

# 3.ID生成

In [6]:
def createId(df,col1,col2):
    df['ID'] = df.iloc[:,col1]+df.iloc[:,col2]

In [7]:
createId(df1,0,1)
createId(df2,0,1)
createId(df3,0,1)
createId(df4,0,1)
createId(df5,0,1)
createId(df6,0,1)

# 4.合并表

In [8]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)

Index(['Stkcd', 'Accper', 'Typrep', 'F050203B', 'ID'], dtype='object')
Index(['Stkcd', 'Reptdt', 'Stknme', 'dir', 'ID'], dtype='object')
Index(['Stkcd', 'Accper', 'Typrep', 'C002006000', 'ID'], dtype='object')
Index(['Stkcd', 'Accper', 'Typrep', 'F081701B', 'ID'], dtype='object')
Index(['Stkcd', 'Accper', 'Typrep', 'F011201A', 'ID'], dtype='object')
Index(['Stkcd', 'Reptdt', 'Stknme', 'Nindcd', 'Nindnme', 'A001000000', 'ID'], dtype='object')


In [9]:
# 创造空列
def createNoColList(df1,df2):
    colList = [str for str in df2.columns if str not in df1.columns]
    for str in colList:
        df1[str] = None

In [10]:
createNoColList(df6,df1)
createNoColList(df6,df2)
createNoColList(df6,df3)
createNoColList(df6,df4)
createNoColList(df6,df5)

In [11]:
df6.head()

,Stkcd,Reptdt,Stknme,Nindcd,Nindnme,A001000000,ID,Accper,Typrep,F050203B,dir,C002006000,F081701B,F011201A
1,000004,2014-12-31,国农科技,C27,医药制造业,3.38283e+08,0000042014-12-31,None,None,None,None,None,None,None
2,000004,2015-12-31,国农科技,C27,医药制造业,3.98674e+08,0000042015-12-31,None,None,None,None,None,None,None
3,000004,2016-12-31,国农科技,C27,医药制造业,2.23716e+08,0000042016-12-31,None,None,None,None,None,None,None
4,000004,2017-12-31,国农科技,C27,医药制造业,2.68844e+08,0000042017-12-31,None,None,None,None,None,None,None
5,000004,2018-12-31,国农科技,C27,医药制造业,3.51177e+08,0000042018-12-31,None,None,None,None,None,None,None


* 根据ID筛选填充

In [12]:
df1.head()

,Stkcd,Accper,Typrep,F050203B,ID
14889,002107,2014-12-31,A,0.055642,0021072014-12-31
14891,002538,2014-12-31,A,0.047583,0025382014-12-31
14893,002654,2014-12-31,A,0.044426,0026542014-12-31
14895,600593,2014-12-31,A,0.05516,6005932014-12-31
14897,300358,2014-12-31,A,0.124917,3003582014-12-31


In [13]:
# 把ID和索引做成字典方便检索
dt = dict(zip(df6['ID'].values,df6['ID'].index))

In [14]:
# 找出有ID的两个表的索引
def combineDf(df,df1):
    for idx in df1.index:
        id =df1.loc[idx,'ID']
        if id in dt:
            # print(idx)
            # 赋值
            for col in df1.columns:
                df.loc[dt[id],col] = df1.loc[idx,col]
    return df

In [15]:
df6 = combineDf(df6,df1)
df6 = combineDf(df6,df2)
df6 = combineDf(df6,df3)
df6 = combineDf(df6,df4)
df6 = combineDf(df6,df5)

In [16]:
# df6.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\中间表.xlsx')

# 5.空值过滤,ST过滤,范围过滤

In [17]:
# 空值过滤
# df6 = df6.dropna()

In [18]:
df6.head()

,Stkcd,Reptdt,Stknme,Nindcd,Nindnme,A001000000,ID,Accper,Typrep,F050203B,dir,C002006000,F081701B,F011201A
1,000004,2014-12-31,国农科技,C27,医药制造业,3.38283e+08,0000042014-12-31,2014-12-31,A,0.053838,0.333333,11212298,0.107499,0.534754
2,000004,2015-12-31,国农科技,C27,医药制造业,3.98674e+08,0000042015-12-31,2015-12-31,A,0.018609,0.333333,4134756,0.494308,0.590402
3,000004,2016-12-31,国农科技,C27,医药制造业,2.23716e+08,0000042016-12-31,2016-12-31,A,0.127625,0.333333,2.24494e+06,1.38821,0.203456
4,000004,2017-12-31,国农科技,C27,医药制造业,2.68844e+08,0000042017-12-31,2017-12-31,A,0.023925,0.333333,5.8471e+06,-0.518178,0.315247
5,000004,2018-12-31,国农科技,C27,医药制造业,3.51177e+08,0000042018-12-31,2018-12-31,A,-0.069716,0.333333,1.75371e+07,1.64685,0.477966


In [19]:
# ST过滤
df6 = df6[[t.find('ST')==-1 for t in df6.iloc[:,2].values]]

In [20]:
# 范围过滤
def fitterMiAndMx(df,col,rate1,rate2):
    sortL = sorted(df[col].values)
    mi = sortL[int(len(sortL)*rate1)]
    mx = sortL[int(len(sortL)*rate2)]
    return df[[t>mi and t<mx for t in df.loc[:,col].values]]

In [21]:
#df6 = fitterMiAndMx(df6,'A001000000',0.025,0.975)
#df6 = fitterMiAndMx(df6,'F050203B',0.025,0.975)
#df6 = fitterMiAndMx(df6,'dir',0.025,0.975)
#df6 = fitterMiAndMx(df6,'C002006000',0.025,0.975)
#df6 = fitterMiAndMx(df6,'F081701B',0.025,0.975)
#df6 = fitterMiAndMx(df6,'F011201A',0.025,0.975)

In [22]:
# df6.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\完成表.xlsx')

# 二、分表处理(按年份分表,小表取最大值作为代表,其他删除)
* 对分出来的6个表的每个企业取最近投资的行数据(称为最近增量表吧)

In [23]:
df7 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\产融集合(1).xlsx')

In [24]:
# 按年份分表
g = list(df7.groupby('year'))

In [25]:
df2013 = g[0][1]
df2014 = g[1][1]
df2015 = g[2][1]
df2016 = g[3][1]
df2017 = g[4][1]
df2018 = g[5][1]

In [26]:
# 取最大值作为代表,其他删除
def getMaxAndFitter(df,Name,col):
    dt = {}
    # 筛选企业最大值
    for name,val in zip(df.loc[:,Name].values,df.loc[:,col].values):
        if name not in dt:
            dt[name] = val
        else:
            dt[name] = max(dt[name],val)
    # 过滤
    return df[[val>=dt[name] for name,val in zip(df.loc[:,Name].values,df.loc[:,col].values)]]

In [27]:
df2013 = getMaxAndFitter(df2013,'名称','持有数量(万股/份)')
df2014 = getMaxAndFitter(df2014,'名称','持有数量(万股/份)')
df2015 = getMaxAndFitter(df2015,'名称','持有数量(万股/份)')
df2016 = getMaxAndFitter(df2016,'名称','持有数量(万股/份)')
df2017 = getMaxAndFitter(df2017,'名称','持有数量(万股/份)')
df2018 = getMaxAndFitter(df2018,'名称','持有数量(万股/份)')

df2013 = getMaxAndFitter(df2013,'名称','持股比例')
df2014 = getMaxAndFitter(df2014,'名称','持股比例')
df2015 = getMaxAndFitter(df2015,'名称','持股比例')
df2016 = getMaxAndFitter(df2016,'名称','持股比例')
df2017 = getMaxAndFitter(df2017,'名称','持股比例')
df2018 = getMaxAndFitter(df2018,'名称','持股比例')

* 最近增量表:把本年的数据依次合并到上一年即可

In [28]:
def insertAndCombine(df,df1,name):
    tempDf = df1.copy()
    dt = dict(zip(df[name].values,df[name].index))
    dt1 = dict(zip(tempDf[name].values,tempDf[name].index))
    # 本年,上年
    for n in df.loc[:,name]:
        if n in tempDf[name].values:
            # 上年有,替换
            tempDf.loc[dt1[n]] = df.loc[dt[n]]
        else:
            # 追加
            tempDf.append(df.loc[dt[n]])
    return tempDf

In [29]:
df2014 = insertAndCombine(df2014,df2013,'名称')
df2015 = insertAndCombine(df2015,df2014,'名称')
df2016 = insertAndCombine(df2016,df2015,'名称')
df2017 = insertAndCombine(df2017,df2016,'名称')
df2018 = insertAndCombine(df2018,df2017,'名称')

In [30]:
#df2013.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\2013.xlsx')
#df2014.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\2014.xlsx')
#df2015.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\2015.xlsx')
#df2016.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\2016.xlsx')
#df2017.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\2017.xlsx')
#df2018.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\2018.xlsx')

In [31]:
print(df2013.shape,df2014.shape,df2015.shape,df2016.shape,df2017.shape,df2018.shape)

(186, 10) (186, 10) (186, 10) (186, 10) (186, 10) (186, 10)


# 6个小表的持股占比合并到另一个表里
* 年份修改同一修改

In [32]:
# 指定第一列为索引
df6 = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\完成表.xlsx',index_col=0) 

In [33]:
# 去掉.SZ
def removeSt(str):
    return str[0:-3]

In [34]:
df2013['代码'] = df2013['代码'].apply(removeSt)
df2014['代码'] = df2014['代码'].apply(removeSt)
df2015['代码'] = df2015['代码'].apply(removeSt)
df2016['代码'] = df2016['代码'].apply(removeSt)
df2017['代码'] = df2017['代码'].apply(removeSt)
df2018['代码'] = df2018['代码'].apply(removeSt)

In [35]:
# 修改年份
df2013['year'] = '2013-12-31'
df2014['year'] = '2014-12-31'
df2015['year'] = '2015-12-31'
df2016['year'] = '2016-12-31'
df2017['year'] = '2017-12-31'
df2018['year'] = '2018-12-31'

In [36]:
# ID生成
def createId(df,col1,col2):
    df['ID'] = df.loc[:,col1]+df.loc[:,col2]

In [37]:
createId(df2013,'代码','year')
createId(df2014,'代码','year')
createId(df2015,'代码','year')
createId(df2016,'代码','year')
createId(df2017,'代码','year')
createId(df2018,'代码','year')

In [38]:
# 合并持股比例到总表
df6['持股比例'] = None
# 把ID和索引做成字典方便检索
dt = dict(zip(df6['ID'].values,df6['ID'].index))

In [39]:
# 找出有ID的两个表的索引
ttt = 0
def combineDfRate(df,df1,colName):
    for idx in df1.index:
        id =df1.loc[idx,'ID']
        if id in dt:
            print(id)
            global ttt
            ttt+=1
            df.loc[dt[id],colName] = df1.loc[idx,colName]
    return df

In [40]:
df6 = combineDfRate(df6,df2013,'持股比例')
df6 = combineDfRate(df6,df2014,'持股比例')
df6 = combineDfRate(df6,df2015,'持股比例')
df6 = combineDfRate(df6,df2016,'持股比例')
df6 = combineDfRate(df6,df2017,'持股比例')
df6 = combineDfRate(df6,df2018,'持股比例')

0003012014-12-31
0005132014-12-31
0005162014-12-31
0007002014-12-31
0007962014-12-31
0008212014-12-31
0009972014-12-31
0020122014-12-31
0020422014-12-31
0020752014-12-31
0020932014-12-31
0020942014-12-31
0021082014-12-31
0021112014-12-31
0021182014-12-31
0021322014-12-31
0021762014-12-31
0021772014-12-31
0022292014-12-31
0022692014-12-31
0022772014-12-31
0022862014-12-31
0022872014-12-31
0022982014-12-31
0023472014-12-31
0023912014-12-31
0023972014-12-31
0024242014-12-31
0024472014-12-31
0024782014-12-31
0025262014-12-31
0025432014-12-31
0025672014-12-31
0025852014-12-31
0025922014-12-31
0026342014-12-31
0026612014-12-31
0026992014-12-31
3003372014-12-31
6000952014-12-31
6001672014-12-31
6001912014-12-31
6001962014-12-31
6002772014-12-31
6002882014-12-31
6003632014-12-31
6003802014-12-31
6005072014-12-31
6005712014-12-31
6005862014-12-31
6006552014-12-31
6006942014-12-31
6007782014-12-31
6007852014-12-31
6008182014-12-31
6008962014-12-31
6012162014-12-31
0003012015-12-31
0005132015-12-

In [41]:
#df6.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\汇总表持股比例.xlsx')

In [66]:
df6.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\汇总表持股比例未删减版.xlsx')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\WuChuPeng\\Desktop\\其他\\宁静\\汇总表持股比例未删减版.xlsx'

In [67]:
dfidx = pd.read_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\编号(1).xlsx')

In [63]:
stocks = np.append(dfidx.iloc[:,0].values,301)

In [49]:
df6.head()

,Stkcd,Reptdt,Stknme,Nindcd,Nindnme,A001000000,ID,Accper,Typrep,F050203B,dir,C002006000,F081701B,F011201A,持股比例
10,5,2018-12-31,世纪星源,N77,生态保护和环境治理业,3.121423e+09,0000052018-12-31,2018-12-31,A,0.051959,0.357143,1.679095e+07,-0.084271,0.494040,None
16,8,2014-12-31,宝利来,H61,住宿业,6.740280e+08,0000082014-12-31,2014-12-31,A,0.011942,0.428571,2.851638e+06,0.008977,0.112276,None
18,8,2016-12-31,神州高铁,C37,铁路、船舶、航空航天和其它运输设备制造业,8.192657e+09,0000082016-12-31,2016-12-31,A,0.090060,0.428571,6.028932e+07,0.448569,0.246583,None
19,8,2017-12-31,神州高铁,C37,铁路、船舶、航空航天和其它运输设备制造业,1.067832e+10,0000082017-12-31,2017-12-31,A,0.094283,0.428571,5.718175e+07,0.242497,0.326949,None
21,9,2014-12-31,中国宝安,S90,综合,1.472550e+10,0000092014-12-31,2014-12-31,A,0.033352,0.428571,6.434760e+08,0.037654,0.628934,None


In [64]:
# 索引过滤
newDf = df6[[t in stocks for t in df6.iloc[:,0].values]]

In [68]:
newDf.to_excel(r'C:\Users\WuChuPeng\Desktop\其他\宁静\汇总表持股比例96未删减版.xlsx')